# 前言-学习本项目你可以获得什么
- 理论学习：了解MCP Client的基础知识
- 上手实操：了解MCP Client的使用方式
- 上手实操：通过MCP Client连接MCP Server，结合已有Agent，打造一个端云结合的进阶Agent。

# 1. 项目背景
### 1.1 什么是MCP
[Model Context Protocol (MCP)](https://modelcontextprotocol.io/introduction)是Anthropic推出的一个开放协议，提供了一种将 AI 模型连接到不同数据源和工具的标准化方式，可实现应用与外部数据源和工具之间的无缝集成。

MCP基础架构由以下几部分组成：
- MCP Hosts: 希望通过 MCP 访问数据的程序，例如 Claude Desktop、IDE 或 AI 工具
- MCP Clients: 与服务器保持 1:1 连接的协议客户端
- MCP Servers: 通过MCP协议公开特定功能的程序
- Local Data Sources: MCP协议可以安全访问的本机文件、数据库、服务等
- Remote Services: MCP Server可以访问的远程服务

其中MCP Server包含以下几类：
- Resources: 上下文和数据，供用户或人工智能模型使用
- Prompts: 为用户提供模板化的消息和工作流程
- Tools: AI模型调用的工具，使模型能够与外部系统交互，例如查询数据库、调用 API 或执行计算

MCP Server Tools有以下特征：
- 每个工具都由名称唯一标识，并包含描述其架构的元数据
- 被设计为模型控制的，这意味着语言模型可以根据其上下文理解和用户的提示自动发现和调用工具

### 1.2 MCP Client
 Host 内部专门用于与 MCP Server 建立和维持一对一连接的模块。它负责按照 MCP 协议的规范发送请求、接收响应和处理数据。简单来说，MCP Client 是 Host 内部处理 RPC 通信的“代理”，专注于与一个 MCP Server 进行标准化的数据、工具或 prompt 的交换。
 
 MCP Client 更多是一个底层技术术语，是关于 MCP Server 连接到 MCP Host 的底层细节，不用过于区分 MCP Host 和 MCP Client。

# 2. 基础操作-通过MCP Client连接Server
## 2.1 stdio方式连接
以[官方server](https://github.com/modelcontextprotocol/quickstart-resources/blob/main/weather-server-python/weather.py)为例。我们将server代码保存为`weather.py`。这个server包含`get_alerts', 'get_forecast'，两个tool。

In [ ]:
import asyncio
from appbuilder.modelcontextprotocol.client import MCPClient


async def main():
    mcp_client = MCPClient()
    await mcp_client.connect_to_server("./weather.py")
    print(mcp_client.tools)


if __name__ == "__main__":
    loop = asyncio.get_event_loop()
    loop.run_until_complete(main())

运行后，打印日志参考：

Connected to server with tools:['get_alerts', 'get_forecast']

INFO:appbuilder:

Connected to server with tools:['get_alerts', 'get_forecast']
[Tool(name='get_alerts', description='Get weather alerts for a US state.\n\n    Args:\n        state: Two-letter US state code (e.g. CA, NY)\n    ', inputSchema={'properties': {'state': {'title': 'State', 'type': 'string'}}, 'required': ['state'], 'title': 'get_alertsArguments', 'type': 'object'}), Tool(name='get_forecast', description='Get weather forecast for a location.\n\n    Args:\n        latitude: Latitude of the location\n        longitude: Longitude of the location\n    ', inputSchema={'properties': {'latitude': {'title': 'Latitude', 'type': 'number'}, 'longitude': {'title': 'Longitude', 'type': 'number'}}, 'required': ['latitude', 'longitude'], 'title': 'get_forecastArguments', 'type': 'object'})]

# 3. 进阶操作-MCP结合已发布Agent，打造一个端云结合的进阶Agent。
Agent示例效果：查询今天头条消息，并获取其中一个网页，在本地打开。并将网页内容提交到agent进行llm总结。

### 3.1 创建Agent
在AppBuilder界面创建一个包含`百度AI搜索`组件的Agent，并发布。
![创建Agent界面](./image/aisearch_agent.png)

### 3.2 保存MCP Server代码
我们使用[Playwright](https://github.com/blackwhite084/playwright-plus-python-mcp/blob/master/src/playwright_server/server.py)作为MCP Server，将代码保存为`mcp_playwright.py`。

### 3.3 执行Agent。

用户可复用下面的代码，替换为自己的token、app_id。执行Agent。

In [ ]:
import os
import asyncio

import appbuilder
from appbuilder.core.console.appbuilder_client.async_event_handler import (
    AsyncAppBuilderEventHandler,
)

from appbuilder.modelcontextprotocol.client import MCPClient


class MyEventHandler(AsyncAppBuilderEventHandler):
    def __init__(self, mcp_client):
        super().__init__()
        self.mcp_client = mcp_client

    def get_current_weather(self, location=None, unit="摄氏度"):
        return "{} 的温度是 {} {}".format(location, 20, unit)

    async def interrupt(self, run_context, run_response):
        thought = run_context.current_thought
        # 绿色打印
        print("\033[1;31m", "-> Agent 中间思考: ", thought, "\033[0m")

        tool_output = []
        for tool_call in run_context.current_tool_calls:
            tool_res = ""
            if tool_call.function.name == "get_current_weather":
                tool_res = self.get_current_weather(**tool_call.function.arguments)
            else:
                print(
                    "\033[1;32m",
                    "MCP工具名称: {}, MCP参数:{}\n".format(
                        tool_call.function.name, tool_call.function.arguments
                    ),
                    "\033[0m",
                )
                mcp_server_result = await self.mcp_client.call_tool(
                    tool_call.function.name, tool_call.function.arguments
                )
                print("\033[1;33m", "MCP结果: {}\n\033[0m".format(mcp_server_result))
                for i, content in enumerate(mcp_server_result.content):
                    if content.type == "text":
                        tool_res += mcp_server_result.content[i].text
            tool_output.append(
                {
                    "tool_call_id": tool_call.id,
                    "output": tool_res,
                }
            )
        return tool_output

    async def success(self, run_context, run_response):
        print("\n\033[1;34m", "-> Agent 非流式回答: ", run_response.answer, "\033[0m")


async def agent_run(client, mcp_client, query):
    tools = mcp_client.tools

    conversation_id = await client.create_conversation()
    with await client.run_with_handler(
        conversation_id=conversation_id,
        query=query,
        tools=tools,
        event_handler=MyEventHandler(mcp_client),
    ) as run:
        await run.until_done()

### 用户需替换成自己的
os.environ["APPBUILDER_TOKEN"] = (
    "xxxx"
)


async def main():
    #### 用户需替换成自己的app_id
    app_id = "xxx"
    appbuilder_client = appbuilder.AsyncAppBuilderClient(app_id)
    mcp_client = MCPClient()
    await mcp_client.connect_to_server("./mcp_playwright.py")
    print(mcp_client.tools)

    await agent_run(
        appbuilder_client,
        mcp_client,
        "先搜索今日头条新闻，取出其中一个url，在用playwright_nagative打开这个url",
    )

    await appbuilder_client.http_client.session.close()


if __name__ == "__main__":
    loop = asyncio.get_event_loop()
    loop.run_until_complete(main())

运行结果：

打开网页：
![网页](./image/playwright_page.png)

Agent返回：
![answer](./image/llm_answer.png)